In [10]:
import sqlalchemy as db

In [11]:
# sqlite://<nohostname>/<path>
# where <path> is relative:
engine = db.create_engine("sqlite:///restaurant.db")

In [12]:
connection = engine.connect()
metadata = db.MetaData()


In [13]:


ingredient_table = db.Table('Ingredient', metadata,
              db.Column('nom_ingredient', db.String(), primary_key=True),
              db.Column('prix_achat', db.Float(), default=0.0),
              )



In [14]:


pays_table = db.Table('Pays', metadata,
              db.Column('pays', db.String(), primary_key=True),
              )



In [15]:


restaurant_table = db.Table('Restaurant', metadata,
              db.Column('code_postal', db.String(), primary_key=True),
              db.Column('parking', db.Integer()),
              db.Column('espace_enfant', db.Integer()),
              db.Column('pmr', db.Integer()),
              db.Column('borne', db.Integer()),
              db.Column('capacite', db.Integer()),
              db.Column('pays', db.String(), db.ForeignKey(Pays.pays)),
              )



NameError: name 'Pays' is not defined

In [51]:


stock_table = db.Table('Stock', metadata,
              db.Column('id_stock', db.Integer()),
              db.Column('nom_ingredient', db.String()),
              db.Column('code_postal', db.String(10)),
              db.Column('quantite', db.Integer()),
              )



In [ ]:


stock = db.Table('stock', metadata,
              db.Column('id_stock', db.Integer()),
              db.Column('nom_ingredient', db.String()),
              db.Column('code_postal', db.String(10)),
              db.Column('quantite', db.Integer()),
              )



In [ ]:
metadata.create_all(engine)